In [1]:
# importing the pandas module for
# data frame
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




# load the data set into train variable.
train = pd.read_csv('/kaggle/input/trafficvehiclesjunction/traffic.csv')
train = train.drop('ID', axis=1)

# display top 5 values of data set
train.head()



,DateTime,Junction,Vehicles
0,2015-11-01 00:00:00,1,15
1,2015-11-01 01:00:00,1,13
2,2015-11-01 02:00:00,1,10
3,2015-11-01 03:00:00,1,7
4,2015-11-01 04:00:00,1,9


In [2]:
# function to get all data from time stamp

# get date
def get_dom(dt):
	return dt.day

# get week day
def get_weekday(dt):
	return dt.weekday()

# get hour
def get_hour(dt):
	return dt.hour

# get year
def get_year(dt):
	return dt.year

# get month
def get_month(dt):
	return dt.month

# get year day
def get_dayofyear(dt):
	return dt.dayofyear

# get year week
def get_weekofyear(dt):
	return dt.weekofyear


train['DateTime'] = train['DateTime'].map(pd.to_datetime)
train['date'] = train['DateTime'].map(get_dom)
train['weekday'] = train['DateTime'].map(get_weekday)
train['hour'] = train['DateTime'].map(get_hour)
train['month'] = train['DateTime'].map(get_month)
train['year'] = train['DateTime'].map(get_year)
train['dayofyear'] = train['DateTime'].map(get_dayofyear)
train['weekofyear'] = train['DateTime'].map(get_weekofyear)

# display
train.head()


,DateTime,Junction,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-11-01 00:00:00,1,15,1,6,0,11,2015,305,44
1,2015-11-01 01:00:00,1,13,1,6,1,11,2015,305,44
2,2015-11-01 02:00:00,1,10,1,6,2,11,2015,305,44
3,2015-11-01 03:00:00,1,7,1,6,3,11,2015,305,44
4,2015-11-01 04:00:00,1,9,1,6,4,11,2015,305,44


In [3]:
train.to_csv('file1.csv')

In [4]:

# display top 5 values of data set
# train = train[train['Junction'] == 1]
train.head()

,DateTime,Junction,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-11-01 00:00:00,1,15,1,6,0,11,2015,305,44
1,2015-11-01 01:00:00,1,13,1,6,1,11,2015,305,44
2,2015-11-01 02:00:00,1,10,1,6,2,11,2015,305,44
3,2015-11-01 03:00:00,1,7,1,6,3,11,2015,305,44
4,2015-11-01 04:00:00,1,9,1,6,4,11,2015,305,44


In [5]:
# train = train.drop(['Junction'], axis=1)
print(train.columns)

Index(['DateTime', 'Junction', 'Vehicles', 'date', 'weekday', 'hour', 'month',
       'year', 'dayofyear', 'weekofyear'],
      dtype='object')


In [6]:
# there is no use of DateTime module
# so remove it
train = train.drop(['DateTime'], axis=1)

# separating class label for training the data
train1 = train.drop(['Vehicles'], axis=1)

# class label is stored in target
target = train['Vehicles']

print(train1.head())
target.head()


   Junction  date  weekday  hour  month  year  dayofyear  weekofyear
0         1     1        6     0     11  2015        305          44
1         1     1        6     1     11  2015        305          44
2         1     1        6     2     11  2015        305          44
3         1     1        6     3     11  2015        305          44
4         1     1        6     4     11  2015        305          44


0    15
1    13
2    10
3     7
4     9
Name: Vehicles, dtype: int64

In [7]:
target = pd.read_csv('/kaggle/working/file1.csv')

# separating class label for training the data
train1 = train.drop('Vehicles', axis=1)

# class label is stored in target
target = train['Vehicles']

train1.head()
#target.head()


,Junction,date,weekday,hour,month,year,dayofyear,weekofyear
0,1,1,6,0,11,2015,305,44
1,1,1,6,1,11,2015,305,44
2,1,1,6,2,11,2015,305,44
3,1,1,6,3,11,2015,305,44
4,1,1,6,4,11,2015,305,44


In [12]:

# train1 = pd.read_csv('/kaggle/working/train1.csv')
# train1 = train1.fillna(train1.mean())
# target = pd.read_csv('/kaggle/working/target.csv')
#importing Random forest
from sklearn.ensemble import RandomForestRegressor

#defining the RandomForestRegressor
#m1=RandomForestRegressor(
# m1.fit(train1,target)
#testing
# m1.predict([[11,6,0,1,2015,11,2]])


In [18]:
import tensorflow as tf
X_train, X_test, y_train, y_test = train_test_split(train1, target, test_size=0.2, random_state=42)

# Create and train the model
# model = DecisionTreeClassifier(random_state=42)
# Compile the model
m1 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])
m1.compile(optimizer='adam', loss='mean_squared_error')
# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, monitor='val_loss'),  # Early stopping
    tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoint.keras', save_best_only=True),  # Model checkpoint
    tf.keras.callbacks.TensorBoard(log_dir='./logs')  # TensorBoard for visualization
]
# Train the model
hisotry = m1.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2,callbacks=callbacks)
m1 = tf.keras.models.load_model('model_checkpoint.keras')
y_pred = m1.predict(X_test)
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Epoch 1/200


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 644.2958 - val_loss: 429.4688
Epoch 2/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 432.7545 - val_loss: 346.4506
Epoch 3/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 376.0605 - val_loss: 326.5144
Epoch 4/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 312.1759 - val_loss: 267.8470
Epoch 5/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 297.3348 - val_loss: 268.2225
Epoch 6/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 286.1072 - val_loss: 254.1803
Epoch 7/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 265.7949 - val_loss: 286.3222
Epoch 8/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 256.6211 - val_loss: 326.2411
Epoch 9/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 271.5523 - val_loss: 490.7379
Epoch 10/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 274.2224 - val_loss: 368.2037
Epoch 11/200
963/963 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 268.1797 - val_loss: 288.4780
Epoch 12/